# Multiple Inheritance and MRO

**Chapter 4 - Learning Python, 5th Edition**

This notebook explores multiple inheritance in Python: how classes can inherit from more than one parent, how Python resolves method lookups using the C3 linearization algorithm, and how `super()` cooperates across complex hierarchies. We also cover practical patterns like mixins and Django-style class-based views.

## Key Concepts
- **Multiple inheritance**: A class can inherit from two or more parent classes
- **Diamond problem**: Ambiguity when two parents share a common ancestor
- **C3 linearization**: The algorithm Python uses to compute Method Resolution Order (MRO)
- **Cooperative super()**: `super()` follows MRO, not the lexical parent class
- **Mixins**: Small, focused classes that add reusable behavior through composition

## Section 1: Basic Multiple Inheritance

A class can inherit from multiple parents. Python searches for attributes and methods by traversing the parent classes in a specific order defined by the MRO.

In [ ]:
# A class inheriting from two independent parents
class DatabaseConnection:
    """Handles database connectivity."""

    def __init__(self, connection_string: str) -> None:
        self.connection_string: str = connection_string
        self.connected: bool = False

    def connect(self) -> str:
        self.connected = True
        return f"Connected to {self.connection_string}"

    def disconnect(self) -> str:
        self.connected = False
        return "Disconnected"


class QueryBuilder:
    """Builds SQL queries."""

    def __init__(self, table: str) -> None:
        self.table: str = table

    def select(self, *columns: str) -> str:
        cols: str = ", ".join(columns) if columns else "*"
        return f"SELECT {cols} FROM {self.table}"

    def insert(self, **values: str) -> str:
        cols: str = ", ".join(values.keys())
        vals: str = ", ".join(f"'{v}'" for v in values.values())
        return f"INSERT INTO {self.table} ({cols}) VALUES ({vals})"


class DatabaseManager(DatabaseConnection, QueryBuilder):
    """Combines connection management with query building."""

    def __init__(self, connection_string: str, table: str) -> None:
        DatabaseConnection.__init__(self, connection_string)
        QueryBuilder.__init__(self, table)

    def execute_query(self, query: str) -> str:
        if not self.connected:
            return "Error: not connected"
        return f"Executing: {query}"


db: DatabaseManager = DatabaseManager("postgresql://localhost/mydb", "users")

print(f"db.connect(): {db.connect()}")
print(f"db.select('name', 'email'): {db.select('name', 'email')}")
print(f"db.insert(name='Alice', email='alice@example.com'):")
print(f"  {db.insert(name='Alice', email='alice@example.com')}")
print(f"db.execute_query(db.select()): {db.execute_query(db.select())}")

# Attribute lookup: DatabaseManager -> DatabaseConnection -> QueryBuilder -> object
print(f"\nMRO: {[cls.__name__ for cls in DatabaseManager.__mro__]}")

## Section 2: The Diamond Problem

The diamond problem occurs when a class inherits from two parents that share a common ancestor. Without careful resolution, the shared ancestor's methods could be called multiple times or ambiguously.

In [ ]:
# Classic diamond: A at the top, B and C in the middle, D at the bottom
#       A
#      / \
#     B   C
#      \ /
#       D

class A:
    """Common ancestor."""

    def process(self) -> str:
        return "A.process"

    def identify(self) -> str:
        return "I am A"


class B(A):
    """Left branch of the diamond."""

    def process(self) -> str:
        return "B.process"


class C(A):
    """Right branch of the diamond."""

    def process(self) -> str:
        return "C.process"


class D(B, C):
    """Bottom of the diamond, inherits from both B and C."""
    pass


d: D = D()

# Which process() wins? B, because B is listed first in D(B, C)
print(f"d.process(): {d.process()}")

# identify() is only defined in A, so it is found through the MRO
print(f"d.identify(): {d.identify()}")

# The MRO shows exactly how Python resolves this
print(f"\nD's MRO: {[cls.__name__ for cls in D.__mro__]}")
print("Order: D -> B -> C -> A -> object")
print("Note: A appears only ONCE, after both B and C")

# Verify that D is an instance of all classes in the hierarchy
print(f"\nisinstance(d, B): {isinstance(d, B)}")
print(f"isinstance(d, C): {isinstance(d, C)}")
print(f"isinstance(d, A): {isinstance(d, A)}")

## Section 3: C3 Linearization Algorithm

Python uses the **C3 linearization** algorithm (also called C3 superclass linearization) to compute the MRO. It guarantees:
1. Children are checked before parents
2. Parent order from the class definition is preserved
3. A valid monotonic ordering exists (or Python raises `TypeError`)

The algorithm merges the linearizations of parents while respecting local precedence order.

In [ ]:
# Step-by-step C3 linearization for a complex hierarchy
#
#         O (object)
#        /|\
#       A  B  C
#       |\ |  |
#       | \| /
#       D  E
#        \ |
#         F

class A:
    pass

class B:
    pass

class C:
    pass

class D(A):
    pass

class E(A, B, C):
    pass

class F(D, E):
    pass


# Python computes the MRO using C3 linearization
# Let's trace it step by step:
# L[A] = A, O
# L[B] = B, O
# L[C] = C, O
# L[D] = D + merge(L[A], A) = D + merge((A, O), A) = D, A, O
# L[E] = E + merge(L[A], L[B], L[C], A, B, C)
#       = E + merge((A,O), (B,O), (C,O), A, B, C)
#       = E, A + merge((O), (B,O), (C,O), B, C)   -- take A (head, not in tail)
#       = E, A, B + merge((O), (O), (C,O), C)      -- take B
#       = E, A, B, C + merge((O), (O), (O))         -- take C
#       = E, A, B, C, O
# L[F] = F + merge(L[D], L[E], D, E)
#       = F + merge((D,A,O), (E,A,B,C,O), D, E)
#       = F, D + merge((A,O), (E,A,B,C,O), E)      -- take D
#       = F, D, E + merge((A,O), (A,B,C,O))         -- skip A (in tail of 2nd), take E
#       = F, D, E, A + merge((O), (B,C,O))           -- take A
#       = F, D, E, A, B + merge((O), (C,O))          -- take B
#       = F, D, E, A, B, C + merge((O), (O))         -- take C
#       = F, D, E, A, B, C, O

print("Computed MRO for each class:")
for cls in [A, B, C, D, E, F]:
    mro_names: list[str] = [c.__name__ for c in cls.__mro__]
    print(f"  L[{cls.__name__}] = {' -> '.join(mro_names)}")

# Verify our manual trace matches Python's computation
print(f"\n__mro__ attribute: {F.__mro__}")
print(f"mro() method:      {F.mro()}")
print(f"Both return the same result: {list(F.__mro__) == F.mro()}")

## Section 4: super() in Multiple Inheritance

`super()` does not simply call the lexical parent class. It calls the **next class in the MRO**. This enables cooperative multiple inheritance, where each class in the chain calls `super()` to ensure every class gets initialized exactly once.

In [ ]:
# Cooperative super() chain in a diamond hierarchy
class Base:
    """Root of the hierarchy."""

    def __init__(self, **kwargs) -> None:
        # Base absorbs remaining kwargs so the chain terminates cleanly
        print(f"  Base.__init__(kwargs={kwargs})")


class Authenticator(Base):
    """Handles user authentication."""

    def __init__(self, username: str = "", **kwargs) -> None:
        print(f"  Authenticator.__init__(username={username!r})")
        self.username: str = username
        super().__init__(**kwargs)  # Passes remaining kwargs along the MRO


class Authorizer(Base):
    """Handles permission checks."""

    def __init__(self, role: str = "guest", **kwargs) -> None:
        print(f"  Authorizer.__init__(role={role!r})")
        self.role: str = role
        super().__init__(**kwargs)  # Passes remaining kwargs along the MRO


class AdminUser(Authenticator, Authorizer):
    """An admin user with both authentication and authorization."""

    def __init__(self, username: str, role: str = "admin") -> None:
        print(f"  AdminUser.__init__(username={username!r}, role={role!r})")
        super().__init__(username=username, role=role)


print("MRO:", [cls.__name__ for cls in AdminUser.__mro__])
print("\nCreating AdminUser:")
admin: AdminUser = AdminUser(username="admin_alice", role="superadmin")

print(f"\nadmin.username: {admin.username}")
print(f"admin.role: {admin.role}")

# The call chain follows the MRO:
# AdminUser.__init__ -> Authenticator.__init__ -> Authorizer.__init__ -> Base.__init__
# NOT AdminUser -> Authenticator -> Base (which would skip Authorizer)

In [ ]:
# Demonstrating that super() follows MRO, not the lexical parent
class X:
    def who_am_i(self) -> str:
        return "X"

class Y(X):
    def who_am_i(self) -> str:
        # super() here will call the next in MRO, which depends on
        # the actual instance's class, not just Y's parent
        return f"Y -> {super().who_am_i()}"

class Z(X):
    def who_am_i(self) -> str:
        return f"Z -> {super().who_am_i()}"

class W(Y, Z):
    def who_am_i(self) -> str:
        return f"W -> {super().who_am_i()}"


w: W = W()
print(f"w.who_am_i(): {w.who_am_i()}")
print(f"W's MRO: {[cls.__name__ for cls in W.__mro__]}")

# When called on W, Y's super() goes to Z (next in MRO), not X!
# Chain: W -> Y -> Z -> X

# Compare with calling on a plain Y instance
y: Y = Y()
print(f"\ny.who_am_i(): {y.who_am_i()}")
print(f"Y's MRO: {[cls.__name__ for cls in Y.__mro__]}")
# Chain: Y -> X (Z is not in Y's MRO)

## Section 5: Mixin Classes

Mixins are small, focused classes designed to be combined with other classes through multiple inheritance. They add specific, reusable behavior without being standalone entities. By convention, mixin class names end with `Mixin`.

In [ ]:
import json
from datetime import datetime


class SerializableMixin:
    """Adds JSON serialization capability to any class."""

    def to_json(self) -> str:
        data: dict = {}
        for key, value in self.__dict__.items():
            if isinstance(value, datetime):
                data[key] = value.isoformat()
            else:
                data[key] = value
        return json.dumps(data, indent=2)

    @classmethod
    def from_json(cls, json_str: str) -> "SerializableMixin":
        data: dict = json.loads(json_str)
        instance = cls.__new__(cls)
        instance.__dict__.update(data)
        return instance


class LoggableMixin:
    """Adds logging capability to any class."""

    _log: list[str] = []

    def log(self, message: str) -> None:
        entry: str = f"[{datetime.now().strftime('%H:%M:%S')}] {self.__class__.__name__}: {message}"
        self._log.append(entry)

    def get_log(self) -> list[str]:
        return list(self._log)


class ValidatableMixin:
    """Adds validation capability to any class."""

    _validation_rules: dict[str, type] = {}

    def validate(self) -> list[str]:
        errors: list[str] = []
        for attr, expected_type in self._validation_rules.items():
            value = getattr(self, attr, None)
            if value is None:
                errors.append(f"{attr} is required")
            elif not isinstance(value, expected_type):
                errors.append(f"{attr} must be {expected_type.__name__}, got {type(value).__name__}")
        return errors


# Compose mixins into a real model
class UserProfile(SerializableMixin, LoggableMixin, ValidatableMixin):
    """A user profile that can be serialized, logged, and validated."""

    _validation_rules: dict[str, type] = {
        "name": str,
        "email": str,
        "age": int,
    }

    def __init__(self, name: str, email: str, age: int) -> None:
        self._log: list[str] = []  # Instance-level log
        self.name: str = name
        self.email: str = email
        self.age: int = age
        self.log(f"Created profile for {name}")


# Demonstrate the composed class
user: UserProfile = UserProfile("Alice", "alice@example.com", 30)

# Serialization from SerializableMixin
json_str: str = user.to_json()
print(f"Serialized:\n{json_str}")

# Validation from ValidatableMixin
errors: list[str] = user.validate()
print(f"\nValidation errors: {errors}")

# Logging from LoggableMixin
user.log("Profile serialized")
print(f"\nLog entries:")
for entry in user.get_log():
    print(f"  {entry}")

# MRO shows all mixins in the chain
print(f"\nMRO: {[cls.__name__ for cls in UserProfile.__mro__]}")

## Section 6: Method Resolution Conflicts

When two parent classes define the same method, the MRO determines which one wins. You can also call a specific parent's method explicitly. If the C3 algorithm cannot produce a consistent ordering, Python raises a `TypeError`.

In [ ]:
# Conflicting methods: the first parent in the class definition wins
class FileExporter:
    def export(self, data: str) -> str:
        return f"Exporting to file: {data}"

    def format_output(self) -> str:
        return "plain text"


class APIExporter:
    def export(self, data: str) -> str:
        return f"Exporting to API: {data}"

    def format_output(self) -> str:
        return "JSON"


class DualExporter(FileExporter, APIExporter):
    """Inherits from both exporters. FileExporter's methods take precedence."""

    def export_both(self, data: str) -> str:
        # Explicitly call specific parent methods when needed
        file_result: str = FileExporter.export(self, data)
        api_result: str = APIExporter.export(self, data)
        return f"{file_result}\n{api_result}"


exporter: DualExporter = DualExporter()

# Default resolution: FileExporter wins (listed first)
print(f"exporter.export('report'): {exporter.export('report')}")
print(f"exporter.format_output(): {exporter.format_output()}")

# Explicit parent calls bypass MRO
print(f"\nexporter.export_both('report'):")
print(exporter.export_both('report'))

print(f"\nMRO: {[cls.__name__ for cls in DualExporter.__mro__]}")

# Demonstrating an impossible MRO (TypeError)
# If C3 linearization cannot produce a consistent order, Python refuses
print("\n--- Impossible MRO ---")

class P:
    pass

class Q:
    pass

class R(P, Q):  # R says: P before Q
    pass

class S(Q, P):  # S says: Q before P (contradicts R)
    pass

try:
    # T(R, S) is impossible: R requires P before Q, S requires Q before P
    exec("""
class T(R, S):
    pass
""")
except TypeError as e:
    print(f"TypeError: {e}")
    print("C3 linearization failed: contradictory ordering constraints")

## Section 7: Real-World Pattern: Django-style Class-Based Views

A common real-world application of multiple inheritance and mixins is the class-based view pattern used in frameworks like Django and Flask. Each mixin adds one concern (authentication, caching, template rendering), and the final view class composes them.

In [ ]:
from typing import Any


# --- Base View ---
class View:
    """Base view that handles HTTP request dispatch."""

    def dispatch(self, method: str, **kwargs: Any) -> str:
        handler = getattr(self, method.lower(), self.method_not_allowed)
        return handler(**kwargs)

    def method_not_allowed(self, **kwargs: Any) -> str:
        return "405 Method Not Allowed"


# --- Mixins ---
class AuthMixin:
    """Adds authentication checks before dispatching."""

    required_role: str = "user"
    _current_user_role: str = "guest"  # Simulated for demo

    def dispatch(self, method: str, **kwargs: Any) -> str:
        if self._current_user_role == "guest":
            return "401 Unauthorized: Please log in"
        if self.required_role == "admin" and self._current_user_role != "admin":
            return "403 Forbidden: Admin access required"
        return super().dispatch(method, **kwargs)


class CacheMixin:
    """Adds response caching for GET requests."""

    _cache: dict[str, str] = {}
    cache_timeout: int = 300  # seconds

    def dispatch(self, method: str, **kwargs: Any) -> str:
        if method.upper() == "GET":
            cache_key: str = f"{self.__class__.__name__}:{method}:{kwargs}"
            if cache_key in self._cache:
                return f"[CACHED] {self._cache[cache_key]}"
            response: str = super().dispatch(method, **kwargs)
            self._cache[cache_key] = response
            return response
        return super().dispatch(method, **kwargs)


class TemplateMixin:
    """Adds template rendering capability."""

    template_name: str = "base.html"

    def render(self, context: dict[str, Any]) -> str:
        items: str = ", ".join(f"{k}={v!r}" for k, v in context.items())
        return f"Rendered {self.template_name} with {{{items}}}"


# --- Composed Views ---
class DashboardView(AuthMixin, CacheMixin, TemplateMixin, View):
    """A protected, cached dashboard page."""

    template_name: str = "dashboard.html"
    required_role: str = "user"

    def get(self, **kwargs: Any) -> str:
        context: dict[str, Any] = {
            "title": "Dashboard",
            "stats": {"users": 150, "orders": 42},
        }
        return self.render(context)


class AdminPanelView(AuthMixin, TemplateMixin, View):
    """An admin-only page (no caching for security)."""

    template_name: str = "admin_panel.html"
    required_role: str = "admin"

    def get(self, **kwargs: Any) -> str:
        return self.render({"title": "Admin Panel", "section": "overview"})

    def post(self, **kwargs: Any) -> str:
        return f"Admin action performed: {kwargs}"


# Simulate requests
dashboard: DashboardView = DashboardView()
admin_panel: AdminPanelView = AdminPanelView()

# Unauthenticated user
print("--- Unauthenticated ---")
print(f"GET /dashboard: {dashboard.dispatch('GET')}")

# Simulate login as regular user
print("\n--- Logged in as regular user ---")
dashboard._current_user_role = "user"
admin_panel._current_user_role = "user"

print(f"GET /dashboard: {dashboard.dispatch('GET')}")
print(f"GET /dashboard (cached): {dashboard.dispatch('GET')}")
print(f"GET /admin: {admin_panel.dispatch('GET')}")

# Simulate login as admin
print("\n--- Logged in as admin ---")
admin_panel._current_user_role = "admin"
print(f"GET /admin: {admin_panel.dispatch('GET')}")
print(f"POST /admin: {admin_panel.dispatch('POST', action='reset_cache')}")
print(f"DELETE /admin: {admin_panel.dispatch('DELETE')}")

# Show MRO for both views
print(f"\nDashboardView MRO: {[c.__name__ for c in DashboardView.__mro__]}")
print(f"AdminPanelView MRO: {[c.__name__ for c in AdminPanelView.__mro__]}")

## Summary

### Multiple Inheritance Mechanics
- Python supports multiple inheritance: `class Child(Parent1, Parent2)`
- Method resolution follows the **MRO**, computed via **C3 linearization**
- The MRO ensures each class appears exactly once, children before parents, and respects the declaration order

### The Diamond Problem
- When two parents share a common ancestor, the ancestor appears **once** in the MRO, after both parents
- The first parent listed in the class definition takes precedence for conflicting methods

### super() and Cooperative Inheritance
- `super()` calls the **next class in the MRO**, not necessarily the lexical parent
- Use `**kwargs` to pass unexpected arguments along the `super()` chain
- Every class in a cooperative hierarchy should call `super()` to ensure the full chain executes

### Mixin Best Practices
- Mixins add a single, focused behavior (serialization, logging, validation)
- Name them with a `Mixin` suffix by convention
- List mixins **before** the base class: `class MyView(MixinA, MixinB, View)`
- Keep mixins independent -- they should not depend on each other

### Key Functions and Attributes
- `cls.__mro__`: Tuple of classes in method resolution order
- `cls.mro()`: Method that returns the MRO as a list
- `super()`: Returns a proxy that delegates to the next class in the MRO
- `isinstance(obj, cls)`: Checks against all classes in the MRO